## Importação de Bibliotecas e Configuração Inicial

Esta célula realiza a importação de todas as bibliotecas necessárias para o pipeline de machine learning e define os caminhos dos arquivos:

### Bibliotecas Importadas:

**Utilitários Gerais:**
- `numpy`: Operações numéricas e arrays
- `pandas`: Manipulação de dataframes

**Otimização e Paralelização:**
- `itertools.product`: Geração de combinações de hiperparâmetros
- `joblib.Parallel, delayed`: Processamento paralelo para otimização

**Pré-processamento (sklearn):**
- `SimpleImputer`: Tratamento de valores ausentes
- `StandardScaler`: Normalização de features numéricas
- `OneHotEncoder`: Codificação de variáveis categóricas
- `ColumnTransformer`: Aplicação de transformações específicas por tipo de coluna
- `Pipeline`: Encadeamento de etapas de processamento

**Modelo e Validação:**
- `HistGradientBoostingClassifier`: Modelo de gradient boosting otimizado
- `StratifiedKFold`: Validação cruzada estratificada
- `accuracy_score`: Métrica de avaliação

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from itertools import product
from joblib import Parallel, delayed

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

TRAIN_PATH = 'train.csv'
TEST_PATH = 'test.csv'
SAMPLE_PATH = 'sample_submission.csv'
OUTPUT_PATH = 'submission.csv'

## Funções Auxiliares do Pipeline

### `load_data()`
Carrega os arquivos CSV (train, test, sample) e valida sua existência. Retorna os 3 DataFrames.

### `basic_feature_engineering(df)`
Cria features derivadas:
- Diferenças temporais (`age_first_last_diff`)
- Transformações log (`log_funding_total`)
- Razões (`funding_per_round`)
- Flags binárias (`has_milestones`, `missing_*`)
- Interações entre features (`funding_milestones_interaction`)

### `get_feature_lists(X)`
Separa automaticamente colunas numéricas e categóricas do dataset.

### `build_preprocessor(numeric_cols, cat_cols)`
Cria pipeline sklearn:
- **Numéricas:** Imputação (mediana) → Normalização (StandardScaler)
- **Categóricas:** Imputação (moda) → One-Hot Encoding

### `optimize_threshold_acc(y_true, y_probs)`

Encontra o threshold ótimo que maximiza a acurácia ao converter probabilidades em predições binárias.

**Funcionamento:**
- Testa 101 thresholds de 0.0 a 1.0 (incrementos de 0.01)
- Para cada threshold, converte probabilidades em classes (0 ou 1)
- Calcula a acurácia e mantém o threshold com melhor resultado
- **Retorna:** threshold ótimo (padrão: 0.5 se nenhum melhorar)

**Uso:** Permite ajustar o ponto de corte de classificação além do padrão 0.5, útil para datasets desbalanceados ou quando se busca maximizar acurácia específica.

In [ ]:
# -----------------------------
# Funções
# -----------------------------
def load_data():
    for p in [TRAIN_PATH, TEST_PATH, SAMPLE_PATH]:
        if not os.path.exists(p):
            raise FileNotFoundError(f"Required file not found: {p}")
    train = pd.read_csv(TRAIN_PATH)
    test = pd.read_csv(TEST_PATH)
    sample = pd.read_csv(SAMPLE_PATH)
    return train, test, sample

def basic_feature_engineering(df):
    df = df.copy()
    if 'age_first_funding_year' in df.columns and 'age_last_funding_year' in df.columns:
        df['age_first_last_diff'] = df['age_last_funding_year'].fillna(0) - df['age_first_funding_year'].fillna(0)
    if 'funding_total_usd' in df.columns:
        df['missing_funding_total'] = df['funding_total_usd'].isna().astype(int)
        df['log_funding_total'] = np.log1p(df['funding_total_usd'].fillna(0))
    if 'funding_rounds' in df.columns and 'funding_total_usd' in df.columns:
        df['funding_per_round'] = df['funding_total_usd'] / (df['funding_rounds'].replace(0, np.nan).fillna(1))
    if 'milestones' in df.columns:
        df['has_milestones'] = (df['milestones'] > 0).astype(int)
    for c in ['age_first_funding_year', 'age_last_funding_year',
              'age_first_milestone_year', 'age_last_milestone_year']:
        if c in df.columns:
            df[f'missing_{c}'] = df[c].isna().astype(int)
    if 'funding_per_round' in df.columns and 'has_milestones' in df.columns:
        df['funding_milestones_interaction'] = df['funding_per_round'] * df['has_milestones']
    if 'log_funding_total' in df.columns and 'age_first_last_diff' in df.columns:
        df['log_funding_by_age_diff'] = df['log_funding_total'] / (df['age_first_last_diff'].replace(0,1))
    return df

def get_feature_lists(X):
    numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    return numeric_cols, cat_cols

def build_preprocessor(numeric_cols, cat_cols):
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_cols),
            ('cat', categorical_transformer, cat_cols)
        ],
        remainder='drop'
    )
    return preprocessor

def optimize_threshold_acc(y_true, y_probs):
    best_thresh = 0.5
    best_score = -1
    for t in np.linspace(0, 1, 101):
        preds = (y_probs >= t).astype(int)
        score = accuracy_score(y_true, preds)
        if score > best_score:
            best_score = score
            best_thresh = t
    return best_thresh

## Avaliação de Combinação de Hiperparâmetros

### `evaluate_combo(params, X, y, preprocessor, skf)`

Avalia uma combinação específica de hiperparâmetros usando validação cruzada estratificada.

**Parâmetros testados:**
- `lr`: Learning rate
- `depth`: Profundidade máxima da árvore
- `leaves`: Número máximo de folhas
- `min_leaf`: Mínimo de amostras por folha
- `l2`: Regularização L2

**Processo:**
1. Cria pipeline com preprocessador + HistGradientBoostingClassifier
2. Executa validação cruzada (StratifiedKFold)
3. Para cada fold: treina, prediz probabilidades no conjunto de validação
4. Acumula predições out-of-fold
5. Otimiza threshold nas predições completas
6. Calcula acurácia final com threshold otimizado

**Retorna:** `(score, params, thresh)` - acurácia, parâmetros testados e melhor threshold

**Uso:** Permite busca em grid paralelizável com avaliação robusta via CV.

In [ ]:
def evaluate_combo(params, X, y, preprocessor, skf):
    lr, depth, leaves, min_leaf, l2 = params
    val_preds = np.zeros(len(X))
    pipe = Pipeline([
        ('pre', preprocessor),
        ('clf', HistGradientBoostingClassifier(
            learning_rate=lr,
            max_iter=1000,
            max_depth=depth,
            max_leaf_nodes=leaves,
            min_samples_leaf=min_leaf,
            l2_regularization=l2,
            early_stopping=False,
            random_state=42
        ))
    ])
    for train_idx, val_idx in skf.split(X, y):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr = y.iloc[train_idx]
        pipe.fit(X_tr, y_tr)
        val_preds[val_idx] = pipe.predict_proba(X_val)[:,1]
    thresh = optimize_threshold_acc(y, val_preds)
    score = accuracy_score(y, (val_preds >= thresh).astype(int))
    return (score, params, thresh)

## Função Principal (`main`)

### 1. Carregamento e Preparação
- Carrega datasets (train, test, sample)
- Aplica feature engineering em train e test
- Separa features (X) e target (y)
- Identifica colunas numéricas e categóricas
- Cria preprocessador

### 2. Grid Search Paralelizado
**Hiperparâmetros testados:**
- `learning_rate`: [0.03, 0.05, 0.1]
- `max_depth`: [4, 5, 6]
- `max_leaf_nodes`: [63, 127]
- `min_samples_leaf`: [3, 5, 10]
- `l2_regularization`: [0.0, 0.1]

**Total:** 108 combinações (3×3×2×3×2)

- Gera todas as combinações com `itertools.product`
- Avalia paralelamente com `joblib.Parallel` (usa todos os cores)
- Cada combo usa validação cruzada 5-fold estratificada
- Seleciona melhor combinação por acurácia

### 3. Treinamento Final e Predição
- Treina modelo com melhores hiperparâmetros no dataset completo
- Prediz probabilidades no test set
- Aplica threshold otimizado para classificação binária
- Gera arquivo `submission.csv`

**Saída:** Arquivo de submissão com predições finais

In [ ]:
# -----------------------------
# Main
# -----------------------------
def main():
    train, test, sample = load_data()

    # Feature engineering
    train_fe = basic_feature_engineering(train)
    test_fe = basic_feature_engineering(test)

    # Preparação de dados
    id_col = 'id' if 'id' in train.columns else None
    target_col = 'labels'
    drop_cols = [id_col] if id_col else []
    X = train_fe.drop(columns=drop_cols + [target_col], errors='ignore')
    y = train_fe[target_col].copy()
    X_test = test_fe.drop(columns=[id_col] if id_col else [], errors='ignore')

    # Preprocessamento
    numeric_cols, cat_cols = get_feature_lists(X)
    preprocessor = build_preprocessor(numeric_cols, cat_cols)

    # -----------------------------
    # Grid search completo paralelizado
    # -----------------------------
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    param_grid = {
        'learning_rate': [0.03, 0.05, 0.1],
        'max_depth': [4,5,6],
        'max_leaf_nodes': [63,127],
        'min_samples_leaf': [3,5,10],
        'l2_regularization': [0.0,0.1]
    }

    combos = list(product(
        param_grid['learning_rate'],
        param_grid['max_depth'],
        param_grid['max_leaf_nodes'],
        param_grid['min_samples_leaf'],
        param_grid['l2_regularization']
    ))

    results = Parallel(n_jobs=-1, verbose=10)(
        delayed(evaluate_combo)(params, X, y, preprocessor, skf) for params in combos
    )

    # Seleciona melhor
    best_score, best_params, best_thresh = max(results, key=lambda x: x[0])
    print(f"Melhor configuração encontrada: {best_params} | Accuracy={best_score:.4f} | threshold={best_thresh:.3f}")

    # Treina HGB completo com os melhores hiperparâmetros
    lr, depth, leaves, min_leaf, l2 = best_params
    pipe_hgb_full = Pipeline([
        ('pre', preprocessor),
        ('clf', HistGradientBoostingClassifier(
            learning_rate=lr,
            max_iter=1000,
            max_depth=depth,
            max_leaf_nodes=leaves,
            min_samples_leaf=min_leaf,
            l2_regularization=l2,
            early_stopping=False,
            random_state=42
        ))
    ])
    pipe_hgb_full.fit(X, y)

    # Predições teste
    test_proba = pipe_hgb_full.predict_proba(X_test)[:,1]
    preds_test = (test_proba >= best_thresh).astype(int)

    # Gera submission
    submission = sample.copy()
    label_col = [c for c in submission.columns if c != 'id'][0]
    submission[label_col] = preds_test
    submission.to_csv(OUTPUT_PATH, index=False)
    print(f"Submission salvo em {OUTPUT_PATH} | linhas={len(submission)} | threshold={best_thresh:.3f}")

if __name__ == '__main__':
    main()

Novo melhor F1=0.8412 | params=(0.03, 4, 63, 3, 0.0) | threshold=0.500
Novo melhor F1=0.8426 | params=(0.03, 4, 63, 5, 0.0) | threshold=0.500
Novo melhor F1=0.8472 | params=(0.03, 4, 63, 5, 0.1) | threshold=0.500
Melhor configuração encontrada: (0.03, 4, 63, 5, 0.1) | F1=0.8472
Submission salvo em submission.csv | linhas=277 | threshold=0.500
